## Лабораторная работа №3

### Импорты

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils import shuffle
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

import numpy as np

from IPython.display import SVG

from datetime import datetime

### Загрузка данных

In [2]:
BATCH = 128
datagen = ImageDataGenerator(rescale=1./255)


    
train_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_train/seg_train',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

test_data = datagen.flow_from_directory('/kaggle/input/intel-image-classification/seg_test/seg_test',
                                        target_size=(150, 150),
                                        batch_size=BATCH,
                                        class_mode='categorical',
                                        shuffle=True)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [3]:
train_data.class_indices

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

Зададим общее количество эпох для тестирования конфигураций равное **20**.

In [4]:
EPOCHS = 15

## GPU Info

In [5]:
import subprocess
import pprint

sp = subprocess.Popen(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

out_str = sp.communicate()
out_list = str(out_str[0]).split('\\n')

out_dict = {}

for item in out_list:
    print(item)

b'Wed Dec 11 09:01:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU

In [6]:
def saveModel(model, filename):
    model.summary()
    plot_model(model,to_file=filename,show_shapes=True, expand_nested=True)

Варьируемые параметры:
* Количество фильтров (32, 64, 128)
* Количество свёрточных слоёв (троек, состоящих из свёртки, функции активации и пространственного объединения)
* Вид функции активации

### Конфигурация №1

Начнём со следующей базовой конфигурации с одним свёрточным слоем и числом фильтров равным 3 (в два раза меньше, чем число классов):

Без max pooling после последнего свёрточного слоя:

In [7]:
model = Sequential([
    layers.Conv2D(3, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model1.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 3)       84        
_________________________________________________________________
flatten (Flatten)            (None, 67500)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               17280256  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 17,314,010
Trainable params: 17,314,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 57s 521ms/step - loss: 1.2504 - accuracy: 0.5135 - val_l

С max pooling после последнего свёрточного слоя:

In [8]:
model = Sequential([
    layers.Conv2D(3, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model1_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 3)       84        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16875)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               4320256   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
Total params: 4,354,010
Trainable params: 4,354,010
Non-trainable params: 0
____________________________________________

## Конфигурация №2

Без max pooling после последнего свёрточного слоя:

In [9]:
model = Sequential([
    layers.Conv2D(3, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model2.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 150, 150, 3)       84        
_________________________________________________________________
flatten_2 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               17280256  
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 774       
Total params: 17,314,010
Trainable params: 17,314,010
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 35s 316ms/step - loss: 3.8095 - accuracy: 0.4199 - val

С max pooling после последнего свёрточного слоя:

In [10]:
model = Sequential([
    layers.Conv2D(3, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model2_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 150, 150, 3)       84        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 3)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16875)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               4320256   
_________________________________________________________________
dense_10 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 774       
Total params: 4,354,010
Trainable params: 4,354,010
Non-trainable params: 0
____________________________________________

Анализ результата:
* точность на тестовой выборке на 20% выше, чем лучший результат, полученный в первой лабораторной работе (полносвязные сети);
* тем не менее, есть определённая уверенность, что полученный результат можно улучшить, поскольку наблюдается довольно быстрое переобучение (уже на четвёртой итерации точность на обучающей выборке больше чем на 30% выше, чем на тестовой и почти достигает максимального значения.
* тестовую точность можно улучшить за счёт увеличения количества фильтров в сети как минимум до числа классов и дальше. Это объясняется тем, что полученный на выходе свёрточного слоя трёхмерный тензор можно рассматривать как пиксель исходного изображения и соответствующая ему вероятность принадлежности его к одному из классов. Для такой интерпретации потребуется как минимум 6 фильтров (число классов в задаче). Проверим нашу теорию в следующей конфигурации.
---

## Конфигурация №3

Без max pooling после последнего свёрточного слоя:

In [11]:
model = Sequential([
    layers.Conv2D(6, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model3.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 150, 150, 6)       168       
_________________________________________________________________
flatten_4 (Flatten)          (None, 135000)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               34560256  
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 774       
Total params: 34,594,094
Trainable params: 34,594,094
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 35s 315ms/step - loss: 2.6383 - accuracy: 0.5014 - val

С max pooling после последнего свёрточного слоя:

In [12]:
model = Sequential([
    layers.Conv2D(6, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model3_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 150, 150, 6)       168       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 6)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 33750)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               8640256   
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 774       
Total params: 8,674,094
Trainable params: 8,674,094
Non-trainable params: 0
____________________________________________

## Конфигурация №4

Без max pooling после последнего свёрточного слоя:

In [13]:
model = Sequential([
    layers.Conv2D(6, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model4.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 150, 150, 6)       168       
_________________________________________________________________
flatten_6 (Flatten)          (None, 135000)            0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               34560256  
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 774       
Total params: 34,594,094
Trainable params: 34,594,094
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 36s 323ms/step - loss: 13.4933 - accuracy: 0.1567 - va

С max pooling после последнего свёрточного слоя:

In [14]:
model = Sequential([
    layers.Conv2D(6, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model4_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 150, 150, 6)       168       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 6)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 33750)             0         
_________________________________________________________________
dense_21 (Dense)             (None, 256)               8640256   
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 774       
Total params: 8,674,094
Trainable params: 8,674,094
Non-trainable params: 0
____________________________________________

### Анализ результатов
* мы улучшили точность на тестовой выборке почти на 10 процентов, это радует
* удалось снизить скорость переобучения, так на четвёртой итерации разница между точностью на обучающей и тестовой выборке сократилась на 10% по сравнению с прошлой конфигурацией
* наша теория о том, что число фильтров должно быть как минимум равно числу классов подтвердилась и дала закономерную прибавку в точности
---

## Конфигурация 5
Продолжим увеличивать число фильтров и оставим пока всё так же один свёрточный слой. Гипотеза: заметной прибавки в тестовой точности это не даст.

Без max pooling после последнего свёрточного слоя:

In [15]:
model = Sequential([
    layers.Conv2D(12, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model5.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 150, 150, 12)      336       
_________________________________________________________________
flatten_8 (Flatten)          (None, 270000)            0         
_________________________________________________________________
dense_24 (Dense)             (None, 256)               69120256  
_________________________________________________________________
dense_25 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_26 (Dense)             (None, 6)                 774       
Total params: 69,154,262
Trainable params: 69,154,262
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 36s 327ms/step - loss: 13.1451 - accuracy: 0.1776 - va

С max pooling после последнего свёрточного слоя:

In [16]:
model = Sequential([
    layers.Conv2D(12, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model5_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 12)      336       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 12)        0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 256)               17280256  
_________________________________________________________________
dense_28 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_29 (Dense)             (None, 6)                 774       
Total params: 17,314,262
Trainable params: 17,314,262
Non-trainable params: 0
__________________________________________

## Конфигурация №6

Без max pooling после последнего свёрточного слоя:

In [17]:
model = Sequential([
    layers.Conv2D(12, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model6.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 150, 150, 12)      336       
_________________________________________________________________
flatten_10 (Flatten)         (None, 270000)            0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               69120256  
_________________________________________________________________
dense_31 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_32 (Dense)             (None, 6)                 774       
Total params: 69,154,262
Trainable params: 69,154,262
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
110/110 [==============================] - 35s 318ms/step - loss: 13.4058 - accuracy: 0.1614 - v

С max pooling после последнего свёрточного слоя:

In [18]:
model = Sequential([
    layers.Conv2D(12, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model6_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 150, 150, 12)      336       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 75, 12)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 67500)             0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               17280256  
_________________________________________________________________
dense_34 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_35 (Dense)             (None, 6)                 774       
Total params: 17,314,262
Trainable params: 17,314,262
Non-trainable params: 0
_________________________________________

### Анализ результатов:
* Как и ожидалось, сколь-нибудь ощутимой прибавки в точности на тестовой выборке по сравнению с предыдущей конфигурацией мы не получили
* Дальнейшее увеличение числа фильтров при одном свёрточном слое особо смысла не имеет: используя 12 фильтров вместо 6 (число классов), мы начинаем классифицировать пиксели уже не на 6 классов, а на 12, но данных для такой классификации недостаточно (поскольку в действительности у нас всего 6 классов).
---

## Конфигурация №7

Введём ещё один свёрточный слой. Первый слой будет иметь 32 фильтра, второй - 6 фильтров. Такая "двухуровневая" свёртка имеет следующую интерпретацию: сперва мы попробуем распознать более сложные элементы изображения, а затем на основании полученного выхода проведём уже привычную свёртку с 6 фильтрами, выход который можно будет интерпретировать так: пикселю исходного изображение и соответствующей ему вероятности принадлежности к одному из классов. Если сойдутся звёзды, мы получим рост точности.

Без max pooling после последнего свёрточного слоя:

In [19]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(6, (3, 3), activation = 'relu', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model7.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 75, 75, 6)         1734      
_________________________________________________________________
flatten_12 (Flatten)         (None, 33750)             0         
_________________________________________________________________
dense_36 (Dense)             (None, 256)               8640256   
_________________________________________________________________
dense_37 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_38 (Dense)             (None, 6)               

С max pooling после последнего свёрточного слоя:

In [20]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(6, (3, 3), activation = 'relu', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model7_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 75, 75, 6)         1734      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 37, 37, 6)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 8214)              0         
_________________________________________________________________
dense_39 (Dense)             (None, 256)               2103040   
_________________________________________________________________
dense_40 (Dense)             (None, 128)             

## Конфигурация №8

Без max pooling после последнего свёрточного слоя:

In [21]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(6, (3, 3), activation = 'tanh', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model8.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 75, 75, 6)         1734      
_________________________________________________________________
flatten_14 (Flatten)         (None, 33750)             0         
_________________________________________________________________
dense_42 (Dense)             (None, 256)               8640256   
_________________________________________________________________
dense_43 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_44 (Dense)             (None, 6)               

С max pooling после последнего свёрточного слоя:

In [22]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(6, (3, 3), activation = 'tanh', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model8_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 75, 75, 6)         1734      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 37, 37, 6)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 8214)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 256)               2103040   
_________________________________________________________________
dense_46 (Dense)             (None, 128)             

---

### Конфигурация №9

Увеличим число фильтров во второй свёртке до 32.

Без max pooling после последнего свёрточного слоя:

In [23]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model9.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
flatten_16 (Flatten)         (None, 180000)            0         
_________________________________________________________________
dense_48 (Dense)             (None, 256)               46080256  
_________________________________________________________________
dense_49 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_50 (Dense)             (None, 6)               

С max pooling после последнего свёрточного слоя:

In [24]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model9_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 43808)             0         
_________________________________________________________________
dense_51 (Dense)             (None, 256)               11215104  
_________________________________________________________________
dense_52 (Dense)             (None, 128)             

### Конфигурация №10

Без max pooling после последнего свёрточного слоя:

In [25]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model10.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
flatten_18 (Flatten)         (None, 180000)            0         
_________________________________________________________________
dense_54 (Dense)             (None, 256)               46080256  
_________________________________________________________________
dense_55 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_56 (Dense)             (None, 6)               

С max pooling после последнего свёрточного слоя:

In [26]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model10_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 43808)             0         
_________________________________________________________________
dense_57 (Dense)             (None, 256)               11215104  
_________________________________________________________________
dense_58 (Dense)             (None, 128)             

## Конфигурация №11

Поскольку переобучение всё так же имеет место быть, попробуем снизить его влияние, добавив ещё один свёрточный слой.

Без max pooling после последнего свёрточного слоя:

In [27]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model11.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 150, 150, 128)     3584      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 75, 75, 64)        73792     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 37, 37, 32)        18464     
_________________________________________________________________
flatten_20 (Flatten)         (None, 43808)             0         
_________________________________________________________________
dense_60 (Dense)             (None, 256)             

С max pooling после последнего свёрточного слоя:

In [28]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model11_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 150, 150, 128)     3584      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 75, 75, 64)        73792     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 37, 37, 32)        18464     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 10368)           

## Конфигурация №12

Без max pooling после последнего свёрточного слоя:

In [29]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model12.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 150, 150, 128)     3584      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 75, 75, 64)        73792     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 37, 37, 32)        18464     
_________________________________________________________________
flatten_22 (Flatten)         (None, 43808)             0         
_________________________________________________________________
dense_66 (Dense)             (None, 256)             

С max pooling после последнего свёрточного слоя:

In [30]:
model = Sequential([
    layers.Conv2D(128, (3, 3), activation = 'tanh', padding = 'same', input_shape = (150, 150, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation = 'tanh', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation = 'tanh', padding = 'same'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(6, activation='softmax')
])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

saveModel(model, 'model12_pooling.png')

time_start = datetime.now()
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), shuffle=True, epochs=EPOCHS, validation_steps=len(test_data), validation_data=test_data)
print('Time: ', datetime.now() - time_start)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 150, 150, 128)     3584      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 75, 75, 64)        73792     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 37, 37, 32)        18464     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 10368)           